In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import erf
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
#%matplotlib tk

In [2]:
result = pd.read_csv("/home/lin/git/summerProject/Heston++onSPXonly_ana_100run")

In [3]:
result[['time', 'iteration', 'pv', 'jac', 'lin_sys', 'e0', 'e*', 'Je', 'Dp', 'count','count2', 'count7']].describe()

,time,iteration,pv,jac,lin_sys,e0,e*,Je,Dp,count,count2,count7
count,100.000000,100.000000,100.000000,100.000000,100.000000,1.000000e+02,1.000000e+02,1.000000e+02,100.000000,100.000000,100.0,100.000000
mean,8.073009,802.540000,931.780000,802.550000,930.780000,1.373096e+06,1.203211e-05,4.154602e-04,0.000181,1.340000,0.0,0.340000
std,33.717237,3289.466115,3981.229856,3289.464239,3981.229856,1.143972e+07,5.780694e-05,1.183564e-03,0.000338,0.901402,0.0,0.901402
min,0.150323,16.000000,17.000000,16.000000,16.000000,9.874506e-04,1.595371e-12,8.066112e-12,0.000000,1.000000,0.0,0.000000
25%,0.379263,40.000000,41.000000,40.000000,40.000000,3.987190e-02,8.075369e-11,5.039480e-05,0.000024,1.000000,0.0,0.000000
50%,0.749229,74.000000,87.500000,74.000000,86.500000,1.465388e-01,9.399302e-11,1.161729e-04,0.000075,1.000000,0.0,0.000000
75%,1.708868,175.250000,186.250000,175.250000,185.250000,2.885694e-01,9.888672e-11,2.757990e-04,0.000185,1.000000,0.0,0.000000
max,293.842522,28279.000000,35644.000000,28279.000000,35643.000000,1.138110e+08,4.667933e-04,9.551195e-03,0.002517,7.000000,0.0,6.000000


In [4]:
result['stopped_by'].unique()
#This show the reasons for stopping:
#1 - stopped by small gradient J^T e
#2 - stopped by small Dp
#3 - stopped by itmax
#4 - singular matrix. Restart from current p with increased \mu
#5 - no further error reduction is possible. Restart with increased mu
#6 - stopped by small ||e||_2
#7 - stopped by invalid (i.e. NaN or Inf) "func" values; a user error

array([6, 5, 1])

In [5]:
#number of solved cases, allowing restart when stopped by 7 or 2
solved = result[result['stopped_by'] == 6]
solved.count().unique()

array([90])

In [6]:
#percentage of solved
solved.count().unique()/result.count().unique()

array([0.9])

In [7]:
#number of solved cases at the first try
solved_first_time = solved[solved['count'] == 1]
solved_first_time.count().unique()

array([76])

In [8]:
#number of cases stopped by small Dp (update of parameter)
DP = result[result['stopped_by'] == 2]
DP.count().unique()

array([0])

In [9]:
#number of cases that stopped by max iteration
itmax = result[result['stopped_by'] == 3]
itmax.count().unique()

array([0])

In [10]:
#number of cases that stopped by mall gradient J^T e
smalJac = result[result['stopped_by'] == 1]
smalJac.count().unique()

array([1])

In [11]:
#average time for all solved cases
solved['time'].describe()['mean']

3.9679969

In [12]:
#average time for cases solved without restart
solved_first_time['time'].describe()['mean']

4.293366894736842

In [13]:
#Average number of iteration for solved cases
solved_first_time['iteration'].describe()['mean']

431.3157894736842

In [14]:
#Average number of iteration for all solved cases
solved['iteration'].describe()['mean']

401.35555555555555

In [15]:
#All solved cases
solved[['Ok', 'Rk', 'Ovbar', 'Rvbar','Ov0', 'Rv0', 'Orho', 'Rrho', 'Osigma', 'Rsigma', 'OI1', 'RI1', 'OI2', 'RI2', 'stopped_by', 'e*']]

,Ok,Rk,Ovbar,Rvbar,Ov0,Rv0,Orho,Rrho,Osigma,Rsigma,OI1,RI1,OI2,RI2,stopped_by,e*
0,1.101351,1.091920,0.460541,0.462785,0.247156,0.247063,-0.642977,-0.643092,0.891420,0.891224,0.000517,0.000519,0.000033,3.457211e-05,6,6.169279e-11
2,2.250448,2.228640,0.739134,0.872436,0.391317,0.468001,-0.153419,-0.140067,0.819242,0.743184,0.003469,0.000125,0.004372,6.884553e-04,6,8.915406e-11
3,4.806764,4.508319,0.386603,0.807836,0.113186,0.241476,-0.299815,-0.204342,0.434073,0.297129,0.007560,0.000414,0.010850,7.098196e-04,6,9.978736e-11
4,2.454574,3.331711,0.401473,0.636128,0.796121,0.772766,-0.490293,-0.481145,0.219788,0.232925,0.000768,0.000902,0.001519,1.420210e-04,6,7.292790e-11
5,3.812073,4.692535,-0.019133,0.243724,1.034419,0.868030,-0.675422,-0.788688,0.489658,0.505360,-0.004266,0.000818,-0.001428,4.622450e-04,6,9.951525e-11
6,2.750371,4.444546,0.537085,0.706773,0.937139,0.694078,-0.789314,-0.665228,0.041102,0.067183,-0.007655,0.000525,-0.005001,6.519387e-05,6,9.757522e-11
7,3.788450,2.815968,0.262987,0.445753,0.282865,0.775985,-0.462269,-0.269215,0.304885,0.188244,0.019543,0.000616,0.016613,8.789995e-07,6,7.181895e-11
8,1.263685,1.460961,0.337060,0.399941,1.039946,0.902791,-0.292119,-0.315372,0.284207,0.305632,-0.004598,0.000784,-0.004591,2.821559e-04,6,8.814260e-11
9,0.961365,0.957368,0.618602,0.621246,0.691744,0.765293,-0.775618,-0.702352,0.621505,0.620087,0.003511,0.000598,0.003024,3.187779e-04,6,4.488197e-11
10,1.059736,1.922033,0.137950,0.525693,0.559959,0.579307,-0.465334,-0.444678,0.375939,0.383204,0.001809,0.000447,0.002846,3.878312e-04,6,9.995363e-11


In [16]:
#next 7 blocks show the average distance between initial guess and real parameter
disIk = np.abs(solved['Ik'] - solved['Rk']).mean()
disIk

1.282020543200164

In [17]:
disIvbar = np.abs(solved['Ivbar'] - solved['Rvbar']).mean()
disIvbar

0.2721187220371466

In [18]:
disIv0 = np.abs(solved['Iv0'] - solved['Rv0']).mean()
disIv0

0.2762051001145511

In [19]:
disIsigma = np.abs(solved['Isigma'] - solved['Rsigma']).mean()
disIsigma

0.3023424069704593

In [20]:
disIrho = np.abs(solved['Irho'] - solved['Rrho']).mean()
disIrho

0.2539841732051876

In [21]:
disII1 = np.abs(solved['II1'] - solved['RI1']).mean()
disII1

0.00033293649515407723

In [22]:
disII2 = np.abs(solved['II2'] - solved['RI2']).mean()
disII2

0.0003037257391168832

In [23]:
#next 7 blocks show the average distance between optimal found and real parameter
disOk = np.abs(solved['Ok'] - solved['Rk']).mean()
disOk

0.49415020186139025

In [24]:
disOvbar = np.abs(solved['Ovbar'] - solved['Rvbar']).mean()
disOvbar

0.12047166967583749

In [25]:
disOsigma = np.abs(solved['Osigma'] - solved['Rsigma']).mean()
disOsigma

0.07521887978501866

In [26]:
disOrho = np.abs(solved['Orho'] - solved['Rrho']).mean()
disOrho

0.10613231651130389

In [27]:
disOv0 = np.abs(solved['Ov0'] - solved['Rv0']).mean()
disOv0

0.0843721771722859

In [28]:
disOI1 = np.abs(solved['OI1'] - solved['RI1']).mean()
disOI1

0.003166987580934268

In [29]:
disOI2 = np.abs(solved['OI2'] - solved['RI2']).mean()
disOI2

0.002699038127278141